In [16]:
import torch
from tensordict import TensorDict

In [17]:
import world_machine

In [18]:
device = "cuda"
torch.set_grad_enabled(False)

In [ ]:
state_size = 6
max_context_size = 200

builder = world_machine.WorldMachineBuilder(state_size,
                                  max_context_size)

builder.add_sensorial_dimension("sensorial", state_size,
                                    torch.nn.Identity(),
                                    torch.nn.Identity())


builder.add_block()
builder.add_block(sensorial_dimension="sensorial")

builder.state_activation = "tanh"


model = builder.build()
model.to(device);
model.eval();

In [20]:
batch_size = 32

In [21]:
x = TensorDict({"state":torch.rand((batch_size, max_context_size, state_size)), "sensorial":torch.rand((batch_size, max_context_size, state_size))})
x = x.to(device)

In [22]:
y = model(state=x["state"], sensorial_data=x)

In [23]:
y = model(state=x["state"], sensorial_data=x, input_sequence_size=1)

In [24]:
for i in range(max_context_size):
    y = model(state=x["state"], sensorial_data=x, input_sequence_size=i+1)

In [25]:
def full():
    for i in range(max_context_size):
        y = model(state=x["state"], sensorial_data=x)

def progressive():
    for i in range(max_context_size):
        y = model(state=x["state"], sensorial_data=x, input_sequence_size=i+1)

full();
progressive();

In [26]:
%timeit full()
%timeit progressive()

612 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
509 ms ± 60.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Base

1.04 s ± 128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)\
928 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# Make continguous

1.07 s ± 116 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)\
909 ms ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

---
# Only Sensorial

637 ms ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)\
468 ms ± 44.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# Only State

612 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)\
509 ms ± 60.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [33]:
637/468, 612/509, 1.07/0.909

(1.3611111111111112, 1.2023575638506876, 1.177117711771177)

In [28]:
def full_1():
    y = model(state=x["state"], sensorial_data=x)

def progressive_1():
    y = model(state=x["state"], sensorial_data=x, input_sequence_size=1)

full_1();
progressive_1();

In [29]:
%timeit full_1()
%timeit progressive_1()

2.94 ms ± 201 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.77 ms ± 111 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [30]:
3.76/3.05

1.2327868852459016